In [1]:
import os, json, time
import pandas as pd

# Import gemini module
import google.generativeai as genai

In [2]:
# !pip install -U google-genai
!pip install numpy pandas
!pip install google-generativeai

In [3]:
data_path = './complete_dataset.csv' ## Segment Transcript
video_segments_path = './video_segments' ## Video Segments
output_dir='./gemini_outputs_video_segments' ##

frame_rate = 0.25  # frames per second
max_frame_size = 512  # Maximum size for frame resizing
temperature = 0.0

GOOGLE_API_KEY  = ''
#model_id = 'models/gemini-1.5-pro-latest'
model_id = 'models/gemini-2.0-flash-001' # Good for context.
model_id = 'models/gemini-1.5-flash-002'
# model_id = 'gemini-2.0-pro-exp-02-05'
model_id = 'models/gemini-1.5-pro-002'

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Research/YoutubePortfolio/Benchmarking

Mounted at /content/drive
/content/drive/MyDrive/Research/YoutubePortfolio/Benchmarking


In [5]:
def extract_json_output(output):
    # Find the starting index of the substring
    # start_index = save_output.find("```json")
    start_index = output.find("```json")

    # Check if the starting substring was found
    if start_index != -1:
        # Adjust start_index to skip the "```json" marker
        start_index += len("```json")

        # Find the ending index starting from the adjusted start_index
        end_index = output.find("```", start_index)

        # Check if the ending substring was found
        if end_index != -1:
            # Extract the substring from the adjusted start_index to the end_index
            result = output[start_index:end_index].strip()
        else:
            result = "Ending marker not found"
    else:
        result = "Starting marker not found"

    return result


def extract_list_output(output):
    # Find the starting index of the substring
    start_index = output.find('[')

    # Check if the starting substring was found
    if start_index != -1:
        # Find the ending index starting from the adjusted start_index
        end_index = output.find(']', start_index)

        # Check if the ending substring was found
        if end_index != -1:
            # Extract the substring from the start_index to end_index + 1 to include the closing bracket
            result = output[start_index:end_index + 1].strip()
        else:
            result = "Ending marker not found for list"
    else:
        result = "Starting marker not found for list"

    return result

def extract_json_or_list(output):
    # First, try to extract JSON from Markdown code blocks
    json_result = extract_json_output(output)

    # Check if JSON was successfully extracted
    if json_result not in ["Starting marker not found", "Ending marker not found"]:
        return json_result
    else:
        # If JSON wasn't found or there was an issue, try to extract a list
        list_result = extract_list_output(output)
        return list_result

In [6]:
def load_gemini_client():
    # client = genai.Client(api_key=GOOGLE_API_KEY)
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel(model_name=model_id)
    return model

def query_gemini(client, prompt, video_file):
    response = client.generate_content(
      [video_file, prompt],
      request_options={"timeout": 800},
      generation_config={"temperature": temperature}
    )
    return extract_json_or_list(response.text)

In [7]:
def segment_create_prompt(series, lm_type = "lm", whole = True, price = False):
    """
    Generates a structured prompt based on the provided title, transcript, language model type

    Args:
        series (pd.Series): The pandas series
        lm_type (str, optional): The type of language model the prompt is intended for (default is "lm").
                                 Options are "lm" or "vlm". vlm has a slightly larger prompt to incorporate facial expression.
        whole (bool, optional): Specifies whether to use the entire transcript (True) or a subset (False).
                                Defaults to True.
        price (bool, optional): Only available for vlm. Can the vlm detect price.

    Returns:
        str: A formatted prompt stored in the variable `prompt`, ready for input into a language model.
    """

    title = series['video_title']

    if whole == True:
        transcript = series['transcript']
    if whole == False:
        segment_transcript = series['segment_transcript']


    if lm_type == "vlm":
        video_statement = ("\nThe video is provided along with this prompt.",)
    else:
        video_statement = ("",)

    if lm_type == "vlm" and whole == True:
        yt_video_statement = "video excerpt, "
    elif lm_type == "vlm" and whole == False:
        yt_video_statement = "video segment, "
    else:
        yt_video_statement = ""


    if lm_type == "vlm":
        facial_expression = ("\n           - Facial Expressions: Neutral or doubtful (furrowed brows, pursed lips).",
                             "\n           - Facial Expressions: Moderate enthusiasm (mild smiles, slightly raised eyebrows).",
                             "\n           - Facial Expressions: Enthusiastic, energetic (wide smiles, raised eyebrows).")
    else:
        facial_expression = ("",
                             "",
                             "")

    if whole == True:
        whole_transcript_specific = (" and video title",
                            f"""Inputs:
    - Video Title: {title}
    - Transcript: {transcript}""",
                            "\n           - Consistency: Low conviction if the title makes a bold claim, but the transcript lacks matching conviction.",
                            "\n           - Consistency: Medium conviction if the title makes a bold claim, followed by consistent confidence in the transcript.",
                            "\n           - Consistency: High conviction if the title and transcript are strongly aligned.")
    # Segment
    else:
        whole_transcript_specific = (", and full-length video title",
                            f"""Inputs:
    - Video Title (Full-Length Video): {title}
        - This is the title of the full-length video from which the transcript segment was extracted.
    - Transcript (Segmented Excerpt, Single Recommendation Only): {segment_transcript}
        - This is a short excerpt from the full-length video, containing a single stock recommendation.
    - Video (Segmented Excerpt, Single Recommendation Only):
        - A related video segment is provided along with this prompt.
""",
                           "\n           - Consistency: Low conviction if the title makes a bold claim, but the transcript lacks matching conviction.",
                            "\n           - Consistency: Medium conviction if the title makes a bold claim, followed by consistent confidence in the transcript.",
                            "\n           - Consistency: High conviction if the title and transcript are strongly aligned.")


    prompt = f"""Analyze the YouTube {yt_video_statement}transcript segment{whole_transcript_specific[0]} of an influencer discussing the US stock market. Identify the single stock recommendation and assess its conviction level.
    {whole_transcript_specific[1]}

    Instructions:
    1. Does the video contain a stock recommendation:
       - Label this as `Stock Recommendation Present` with either "Yes" or "No".

    2. If `Stock Recommendation Present` is "Yes", store the single stock recommendation under the key "Recommendation", formatted as a one-item list. The recommendation should follow this structure:{{"Action": "Buy | Hold | Don't Buy | Sell | Short Sell | Unclear",
         "Justification": "Brief explanation for the action based on the transcript",
         "Conviction Score": "1 | 2 | 3",
         "Ticker Name": "Ticker name"}}

       Details for each field:
        - `Action`: Categorize the recommendation as:
          - "Buy": Purchase shares of the stock.
          - "Hold": Retain the stock if already owned.
          - "Don't Buy": Refrain from purchasing the stock.
          - "Sell": Sell shares currently owned.
          - "Short Sell": Sell shares not currently owned, intending to buy them back later at a lower price.
          - "Unclear": When the action is not explicitly stated.
       - `Justification`: Provide a brief explanation for the action based on the transcript.
       - `Conviction Score`: Assign a score based on the following criteria:
         - "1" (Low Conviction):
           - Tone: Hesitant or uncertain language, frequent qualifiers (e.g., “maybe,” “possibly”).{facial_expression[0]}
           - Delivery: Reserved or doubtful language.{whole_transcript_specific[2]}
         - "2" (Moderate Conviction):
           - Tone: Relatively confident language with some qualifiers.{facial_expression[1]}
           - Delivery: Balanced and moderately positive language.{whole_transcript_specific[3]}
         - "3" (High Conviction):
           - Tone: Strong, assertive language without hesitation.{facial_expression[2]}
           - Delivery: Decisive recommendations with no qualifiers.{whole_transcript_specific[4]}
       - `Ticker Name`: Specify the ticker name of the stock being discussed.

    3. If `Stock Recommendation Present` is "No", return the following structure:{{"Stock Recommendation Present": "No",
         "Recommendation": []
       }}

    Output Requirements:
    - Return only valid JSON that can be directly parsed by JSON libraries.
    - Do not include any additional text, comments, formatting indicators (e.g., `json` or backticks), or explanatory content.
    """

    return prompt

In [8]:
def save_response(video_id, start, end, response, output_dir):
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists
    file_path = os.path.join(output_dir, f"{video_id}__{start}__{end}.txt")
    with open(file_path, "w") as f:
        f.write(response)

def load_existing_response(video_id, start, end, output_dir):
    file_path = os.path.join(output_dir, f"{video_id}__{start}__{end}.txt")
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            return f.read()
    return None

In [9]:
def process_video_row(row, client, frame_rate, max_frame_size, video_segments_path, output_dir):
    video_id = row['video_id']
    title = row['video_title']
    start = row['start']
    end = row['end']
    segment_transcript = row['segment_transcript']

    # Check if output already exists
    existing_response = load_existing_response(video_id, start, end, output_dir)
    if existing_response:
        print(f"Skipping row as output already exists: {video_id}__{start}__{end}")
        return existing_response

    # # Skip if segment duration exceeds 4 minutes
    # if (end - start) > 240:
    #     print(f"Skipping row due to segment duration exceeding 4 minutes: {video_id}__{start}__{end}")
    #     return "VideoTooLong"

    video_file_path = os.path.join(video_segments_path, f"{video_id}__{start}__{end}.mp4")
    if not os.path.exists(video_file_path):
        print(f"Video file not found: {video_file_path}")
        return "NoVideoFile"
    # If using gemini-2.0-pro-exp-02-05, wait 12 seconds to comply with the rate limit
    if model_id == 'gemini-2.0-pro-exp-02-05':
        print("Rate limit in effect for gemini-2.0-pro-exp-02-05. Sleeping for 12 seconds.")
        time.sleep(12)
    try:
        # Create prompt
        prompt = segment_create_prompt(row, lm_type = "vlm", whole = False, price = False)

        video_file = upload_media_and_return_objects(video_file_path)

        # Query Gemini model
        response = query_gemini(client, prompt, video_file)
        genai.delete_file(video_file.name)

        #print(f'Deleted file {video_file.uri}')
        # print("response", response)
        # Save response
        save_response(video_id, start, end, response, output_dir)
        return response

    except Exception as e:
        print(f"Error processing video {video_id}__{start}__{end}: {e}")
        return "ErrorWhileProcessing"

In [10]:
def upload_media_and_return_objects(file_path):
    video_file = genai.upload_file(path=file_path)

    while video_file.state.name == "PROCESSING":
        time.sleep(10)
        video_file = genai.get_file(video_file.name)

    if video_file.state.name == "FAILED":
        raise ValueError(video_file.state.name)
    print(f'Video processing complete: ' + video_file.uri)

    return video_file

In [11]:
def process_segment_videos(data_path, video_segments_path, frame_rate, max_frame_size, output_dir):
    data = pd.read_csv(data_path)

    results = []

    # Load processor and model
    client = load_gemini_client()
    # Iterate over each row

    ## Only going over one video for now
    for index, row in data.iterrows():
        try:
            response = process_video_row(row, client, frame_rate, max_frame_size, video_segments_path, output_dir)
            #print(f"Response for row {index}: {response}")
            results.append(response)
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            results.append(None)

In [12]:
data = pd.read_csv(data_path)
results = []

# Load processor and model
client = load_gemini_client()

# Iterate over each row
for idx, row in data[:1].iterrows():
    # Extract title and transcript
    series = row
    break

In [ ]:
prompt = segment_create_prompt(series = series,
              lm_type = 'vlm',
              whole = False,
              price = False
             )

In [ ]:
print(prompt)

Analyze the YouTube video segment, transcript segment, and full-length video title of an influencer discussing the US stock market. Identify the single stock recommendation and assess its conviction level.
    Inputs:
    - Video Title (Full-Length Video): 5 Stocks to Buy Now to Double Your Money 
        - This is the title of the full-length video from which the transcript segment was extracted.
    - Transcript (Segmented Excerpt, Single Recommendation Only):  these in a minute. First up is $225 million dollar Veritone Inc., ticker V-E-R-I, a cloud-based AI platform that structures audio and video data. And now Nation, if that sounds like a bunch of tech jargon, just understand that Veritone is in the convergence of what's going to be the three biggest trends over the next decade. Data analysis of audio and video content, cloud-based connectivity, and an AI platform that learns to become more effective. And the potential markets for these speak for themselves. The company's AIware p

## Video SEGMENT Data

In [ ]:
data_path = './complete_dataset.csv' ## Segment Transcript
video_segments_path = './video_segments' ## Video Segments
output_dir='./gemini_outputs_video_segments' ##

frame_rate = 0.25  # frames per second
max_frame_size = 512  # Maximum size for frame resizing
temperature = 0.0

In [13]:
"""
def load_gemini_client():
    # client = genai.Client(api_key=GOOGLE_API_KEY)
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel(model_name=model_id)
    return model
"""


data = pd.read_csv(data_path)

# Selected region and only if rec is present. (381 videos)
# Applying all the filters using .loc
filtered_df = data.loc[
    (data['action_source'] == "Selected region") &
    (data['is_rec_present'] == "Yes") &
    (~data['transcript'].isna()) &
    (data['transcript'].str.strip() != "") &
    (data['transcript'].str.split().str.len() >= 3)
]

data = filtered_df.copy()


frame_rate = 0.25  # frames per second
max_frame_size = 512  # Maximum size for frame resizing
temperature = 0.0


model_name = 'gemini-2.0-pro-exp-02-05'
model_name = 'gemini-1.5-pro-002'

model_name = 'gemini-2.0-flash-001' # Good for context.
model_name = 'gemini-1.5-flash-002'

# Lower intelligence tasks
model_name = 'gemini-1.5-flash-8b-001'


## Good

## Happy
# 'gemini-1.5-flash-8b-001'
# 'gemini-1.5-pro-002'

#for model_name in ['gemini-2.0-flash-001', 'gemini-1.5-flash-002',  'gemini-2.0-pro-exp-02-05']:

for model_name in ['gemini-2.0-flash-001', 'gemini-1.5-flash-002', 'gemini-2.0-pro-exp-02-05']:

  model_id = "models/" + model_name

  data_path = './complete_dataset.csv' ## Segment Transcript
  video_segments_path = './video_segments' ## Video Segments
  output_dir=f'./{model_name}_outputs_video_segments' ##


  results = []

  # Load processor and model
  genai.configure(api_key=GOOGLE_API_KEY)
  client = genai.GenerativeModel(model_name=model_id)
  # Iterate over each row

  ## Only going over one video for now
  for index, row in data.iterrows():
    try:
        response = process_video_row(row, client, frame_rate, max_frame_size, video_segments_path, output_dir)
        #print(f"Response for row {index}: {response}")
        results.append(response)
        time.sleep(12)
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        results.append(None)

Skipping row as output already exists: 0CJU8R4oNFk__197.7986688222514__267.77438718612933
Skipping row as output already exists: 0CJU8R4oNFk__515.435925412456__559.6029779831492
Skipping row as output already exists: 0CJU8R4oNFk__560.5183963259649__614.2462458777909
Skipping row as output already exists: 1Gm4A7EFYI4__39.577461060109286__642.9538446765027
Skipping row as output already exists: 1Lx7z_x4Rc0__118.7247700983607__320.404668021858
Skipping row as output already exists: 1Lx7z_x4Rc0__329.91787075409843__432.6604602622952
Skipping row as output already exists: 1Lx7z_x4Rc0__466.90799009836064__519.4208691803278
Skipping row as output already exists: 1P65IOhAFFI__37.801578557377056__57.39668254426229
Skipping row as output already exists: 1P65IOhAFFI__61.56257079344262__85.47785518688524
Skipping row as output already exists: 1Wh-i9i4V28__231.11287449851733__248.4603299693504
Skipping row as output already exists: 1Wh-i9i4V28__250.11016278944484__270.8441434801026
Skipping row as 

ERROR:tornado.access:500 GET /v1beta/files/qi98zpn7rf4?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 593.79ms


Error processing video wniiVmbSAcM__372.6438055947547__375.8268134749292: 500 GET https://generativelanguage.googleapis.com/v1beta/files/qi98zpn7rf4?%24alt=json%3Benum-encoding%3Dint: Failed to convert server response to JSON
Skipping row as output already exists: wniiVmbSAcM__375.8409836332695__383.1580847074286
Skipping row as output already exists: wniiVmbSAcM__383.18648385908574__393.31349554872696
Skipping row as output already exists: XDnFv3YlgqE__120.58249069676988__266.4009496330843
Skipping row as output already exists: XDnFv3YlgqE__266.4863882052647__416.9043211906092
Skipping row as output already exists: XDnFv3YlgqE__416.9086916840727__498.8354596056936
Skipping row as output already exists: XDnFv3YlgqE__498.95673516564415__606.1355484233129
Skipping row as output already exists: XDYTp-Axwog__11.866375074243356__27.228661122838943
Skipping row as output already exists: XDYTp-Axwog__27.28983580206083__47.87348536770836
Skipping row as output already exists: XDYTp-Axwog__47.9

ERROR:tornado.access:500 GET /v1beta/files/9n3nwyp933vj?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 355.47ms


Error processing video wniiVmbSAcM__372.6438055947547__375.8268134749292: 500 GET https://generativelanguage.googleapis.com/v1beta/files/9n3nwyp933vj?%24alt=json%3Benum-encoding%3Dint: Failed to convert server response to JSON
Skipping row as output already exists: wniiVmbSAcM__375.8409836332695__383.1580847074286
Skipping row as output already exists: wniiVmbSAcM__383.18648385908574__393.31349554872696
Skipping row as output already exists: XDnFv3YlgqE__120.58249069676988__266.4009496330843
Skipping row as output already exists: XDnFv3YlgqE__266.4863882052647__416.9043211906092
Skipping row as output already exists: XDnFv3YlgqE__416.9086916840727__498.8354596056936
Skipping row as output already exists: XDnFv3YlgqE__498.95673516564415__606.1355484233129
Skipping row as output already exists: XDYTp-Axwog__11.866375074243356__27.228661122838943
Skipping row as output already exists: XDYTp-Axwog__27.28983580206083__47.87348536770836
Skipping row as output already exists: XDYTp-Axwog__47.

ERROR:tornado.access:500 GET /v1beta/files/jg7d5nkybmdo?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 330.09ms


Error processing video wniiVmbSAcM__372.6438055947547__375.8268134749292: 500 GET https://generativelanguage.googleapis.com/v1beta/files/jg7d5nkybmdo?%24alt=json%3Benum-encoding%3Dint: Failed to convert server response to JSON
Skipping row as output already exists: wniiVmbSAcM__375.8409836332695__383.1580847074286
Skipping row as output already exists: wniiVmbSAcM__383.18648385908574__393.31349554872696
Skipping row as output already exists: XDnFv3YlgqE__120.58249069676988__266.4009496330843
Skipping row as output already exists: XDnFv3YlgqE__266.4863882052647__416.9043211906092
Skipping row as output already exists: XDnFv3YlgqE__416.9086916840727__498.8354596056936
Skipping row as output already exists: XDnFv3YlgqE__498.95673516564415__606.1355484233129
Skipping row as output already exists: XDYTp-Axwog__11.866375074243356__27.228661122838943
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/w4vy2q04t0cq
Skipping row as output already exists: XDYTp-Axwo

In [ ]:
#process_videos(data_path, video_segments_path, frame_rate, max_frame_size, output_dir)

Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/6yz7xw4lrqif
Deleted file https://generativelanguage.googleapis.com/v1beta/files/6yz7xw4lrqif
Response for row 0: {"Stock Recommendations Present": "Yes",
 "Recommendations": [{"Action": "Buy",
   "Justification": "The influencer mentions Veritone Inc. is \"in the convergence of what's going to be the three biggest trends over the next decade\", citing data analysis of audio and video content, cloud-based connectivity, and AI platforms. They also highlight a 244% revenue jump in two years and analyst price targets suggesting a potential 120% return.",
   "Conviction Score": "3",
   "Ticker Name": "VERI"}]}


## Video FULL-LENGTH Data

In [ ]:
def segment_create_prompt(series, lm_type = "lm", whole = True, price = False):
    """
    Generates a structured prompt based on the provided title, transcript, language model type

    Args:
        series (pd.Series): The pandas series
        lm_type (str, optional): The type of language model the prompt is intended for (default is "lm").
                                 Options are "lm" or "vlm". vlm has a slightly larger prompt to incorporate facial expression.
        whole (bool, optional): Specifies whether to use the entire transcript (True) or a subset (False).
                                Defaults to True.
        price (bool, optional): Only available for vlm. Can the vlm detect price.

    Returns:
        str: A formatted prompt stored in the variable `prompt`, ready for input into a language model.
    """

    title = series['video_title']

    if whole == True:
        transcript = series['transcript']
    if whole == False:
        segment_transcript = series['segment_transcript']


    if lm_type == "vlm":
        video_statement = ("\nThe video is provided along with this prompt.",)
    else:
        video_statement = ("",)

    if lm_type == "vlm" and whole == True:
        yt_video_statement = "video excerpt, "
    elif lm_type == "vlm" and whole == False:
        yt_video_statement = "video segment, "
    else:
        yt_video_statement = ""


    if lm_type == "vlm":
        facial_expression = ("\n           - Facial Expressions: Neutral or doubtful (furrowed brows, pursed lips).",
                             "\n           - Facial Expressions: Moderate enthusiasm (mild smiles, slightly raised eyebrows).",
                             "\n           - Facial Expressions: Enthusiastic, energetic (wide smiles, raised eyebrows).")
    else:
        facial_expression = ("",
                             "",
                             "")

    if whole == True:
        whole_transcript_specific = (" and video title",
                            f"""Inputs:
    - Video Title: {title}
    - Transcript: {transcript}""",
                            "\n           - Consistency: Low conviction if the title makes a bold claim, but the transcript lacks matching conviction.",
                            "\n           - Consistency: Medium conviction if the title makes a bold claim, followed by consistent confidence in the transcript.",
                            "\n           - Consistency: High conviction if the title and transcript are strongly aligned.")
    # Segment
    else:
        whole_transcript_specific = (", and full-length video title",
                            f"""Inputs:
    - Video Title (Full-Length Video): {title}
        - This is the title of the full-length video from which the transcript segment was extracted.
    - Transcript (Segmented Excerpt, Single Recommendation Only): {segment_transcript}
        - This is a short excerpt from the full-length video, containing a single stock recommendation.
    - Video (Segmented Excerpt, Single Recommendation Only):
        - A related video segment is provided along with this prompt.
""",
                           "\n           - Consistency: Low conviction if the title makes a bold claim, but the transcript lacks matching conviction.",
                            "\n           - Consistency: Medium conviction if the title makes a bold claim, followed by consistent confidence in the transcript.",
                            "\n           - Consistency: High conviction if the title and transcript are strongly aligned.")


    prompt = f"""Analyze the YouTube {yt_video_statement}transcript segment{whole_transcript_specific[0]} of an influencer discussing the US stock market. Identify the single stock recommendation and assess its conviction level.
    {whole_transcript_specific[1]}

    Instructions:
    1. Does the video contain a stock recommendation:
       - Label this as `Stock Recommendation Present` with either "Yes" or "No".

    2. If `Stock Recommendation Present` is "Yes", store the single stock recommendation under the key "Recommendation", formatted as a one-item list. The recommendation should follow this structure:{{"Action": "Buy | Hold | Don't Buy | Sell | Short Sell | Unclear",
         "Justification": "Brief explanation for the action based on the transcript",
         "Conviction Score": "1 | 2 | 3",
         "Ticker Name": "Ticker name"}}

       Details for each field:
        - `Action`: Categorize the recommendation as:
          - "Buy": Purchase shares of the stock.
          - "Hold": Retain the stock if already owned.
          - "Don't Buy": Refrain from purchasing the stock.
          - "Sell": Sell shares currently owned.
          - "Short Sell": Sell shares not currently owned, intending to buy them back later at a lower price.
          - "Unclear": When the action is not explicitly stated.
       - `Justification`: Provide a brief explanation for the action based on the transcript.
       - `Conviction Score`: Assign a score based on the following criteria:
         - "1" (Low Conviction):
           - Tone: Hesitant or uncertain language, frequent qualifiers (e.g., “maybe,” “possibly”).{facial_expression[0]}
           - Delivery: Reserved or doubtful language.{whole_transcript_specific[2]}
         - "2" (Moderate Conviction):
           - Tone: Relatively confident language with some qualifiers.{facial_expression[1]}
           - Delivery: Balanced and moderately positive language.{whole_transcript_specific[3]}
         - "3" (High Conviction):
           - Tone: Strong, assertive language without hesitation.{facial_expression[2]}
           - Delivery: Decisive recommendations with no qualifiers.{whole_transcript_specific[4]}
       - `Ticker Name`: Specify the ticker name of the stock being discussed.

    3. If `Stock Recommendation Present` is "No", return the following structure:{{"Stock Recommendation Present": "No",
         "Recommendation": []
       }}

    Output Requirements:
    - Return only valid JSON that can be directly parsed by JSON libraries.
    - Do not include any additional text, comments, formatting indicators (e.g., `json` or backticks), or explanatory content.
    """

    return prompt

In [ ]:
data = pd.read_csv(data_path)

# Selected region and only if rec is present. (381 videos)
# Applying all the filters using .loc
filtered_df = data.loc[
    (data['action_source'] == "Selected region") &
    (data['is_rec_present'] == "Yes") &
    (~data['transcript'].isna()) &
    (data['transcript'].str.strip() != "") &
    (data['transcript'].str.split().str.len() >= 3)
]

data = filtered_df.copy().head(1)

In [ ]:
df = pd.read_csv("../data_needed_for_inference/complete_dataset.csv")

for idx, row in df.iterrows():
    # Extract title and transcript
    series = row
    break

In [ ]:
prompt = whole_create_prompt(series = series,
              lm_type = 'vlm',
              whole = True,
              price = False
             )

In [ ]:
data = pd.read_csv(data_path)

# Selected region and only if rec is present. (381 videos)
# Applying all the filters using .loc
filtered_df = data.loc[
    (data['action_source'] == "Selected region") &
    (data['is_rec_present'] == "Yes") &
    (~data['transcript'].isna()) &
    (data['transcript'].str.strip() != "") &
    (data['transcript'].str.split().str.len() >= 3)
]

data = filtered_df.copy()


frame_rate = 0.25  # frames per second
max_frame_size = 512  # Maximum size for frame resizing
temperature = 0.0


model_name = 'gemini-2.0-pro-exp-02-05'
model_name = 'gemini-1.5-pro-002'

model_name = 'gemini-2.0-flash-001' # Good for context.
model_name = 'gemini-1.5-flash-002'

# Lower intelligence tasks
model_name = 'gemini-1.5-flash-8b-001'


## Good

## Happy
# 'gemini-1.5-flash-8b-001'
# 'gemini-1.5-pro-002'

#for model_name in ['gemini-2.0-flash-001', 'gemini-1.5-flash-002',  'gemini-2.0-pro-exp-02-05']:

for model_name in ['gemini-2.0-flash-001', 'gemini-1.5-flash-002', 'gemini-2.0-pro-exp-02-05']:

  model_id = "models/" + model_name

  data_path = './complete_dataset.csv' ## Segment Transcript
  video_segments_path = './video_segments' ## Video Segments
  output_dir=f'./{model_name}_outputs_video_segments' ##


  results = []

  # Load processor and model
  genai.configure(api_key=GOOGLE_API_KEY)
  client = genai.GenerativeModel(model_name=model_id)
  # Iterate over each row

  ## Only going over one video for now
  for index, row in data.iterrows():
    try:
        response = process_video_row(row, client, frame_rate, max_frame_size, video_segments_path, output_dir)
        #print(f"Response for row {index}: {response}")
        results.append(response)
        time.sleep(12)
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        results.append(None)